## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [2]:
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup

I imported in a dataset provided by IMDB's interface. Scraping the data from their website turned out to be unnecessary.

In [2]:
name_basics_df = pd.read_csv('zippedData/name.basics.tsv', sep='\t')
title_akas_df = pd.read_csv('zippedData/title.akas.tsv', sep='\t')
title_basics_df = pd.read_csv('zippedData/title.basics.tsv', sep='\t')
title_crew_df = pd.read_csv('zippedData/title.crew.tsv', sep='\t')
title_principals_df = pd.read_csv('zippedData/title.principals.tsv', sep='\t')
title_ratings_df = pd.read_csv('zippedData/title.ratings.tsv', sep='\t')

D:\Python\anaconda37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Python\anaconda37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type int64

This database is still missing the cost to make movie and the profit. We found a database containing the production costs, worldwide and domestic gross income. This information is not given to us in a file unfortunately like the previous dataset was, so we will have to employ some datascraping to pull this data off the website. We are grabbing our data from https://www.the-numbers.com/movie/budgets/all
I defined 2 functions to ease the load since I need to run a separate iteration of this script in order to catch the first page due to it having a unique url. 

In [85]:
'''Takes in the URL as an argument, and returns the chart that the movies are in.'''
def html_parse(url):
    html_page = requests.get(url, timeout=5)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    chart = soup.find(id = 'page_filling_chart')
    movie_container = chart.findAll('tr') #grabs all movie entries on the page and puts into a list
    return movie_container

In [86]:
'''Takes in the row containing the movie title, profits, release date, and costs and returns a list of those.'''
def movie_extract(data_container):
        release_date = data_container[1].text #release date
        movie_name = data_container[2].text #name of movie
        prod_budget = data_container[3].get_text(strip=True) #production budget
        gross_dom = data_container[4].get_text(strip=True) #gross domestic 
        gross_world = data_container[5].get_text(strip=True) #gross worldwide
        
        return [release_date, movie_name, prod_budget, gross_dom, gross_world]

This is the initial run of the function due to the URL being unique for the very first page.

In [87]:
df = pd.DataFrame(columns = ['Release_Date', 'Movie_Name', 'Production_Budget', 'Gross_Domestic', 'Gross_Worldwide'])
url = "https://www.the-numbers.com/movie/budgets/all"
movie_container = html_parse(url)

#for loop here to iterate through the list of movies in movie_container
for u in range(1,101): #this'll break on the last page, find better way to do it
    data_container = movie_container[u].findAll('td')

    data_series = movie_extract(data_container)
    
    a_series = pd.Series(data_series, index = df.columns)
    df = df.append(a_series, ignore_index=True)

Iterates through all the pages utilizing URL hacking to find all of the pages.

In [88]:
for i in range(2,61): #change 4 to 61 for all
    url = "https://www.the-numbers.com/movie/budgets/all/{}01".format(i)
    movie_container = html_parse(url)

    #for loop here to iterate through the list of movies in movie_container
    for u in range(1,101): 
        if i == 60 and u == 44:
            break
        data_container = movie_container[u].findAll('td')

        data_series = movie_extract(data_container)
        
        a_series = pd.Series(data_series, index = df.columns)
        df = df.append(a_series, ignore_index=True)


In [90]:
df.to_csv('zippedData/movieMoney.csv') #saving it to a CSV so I don't have to scrape everytime I want to look at the data

In [4]:
df

,Release_Date,Movie_Name,Production_Budget,Gross_Domestic,Gross_Worldwide
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,396,099,202"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,065,478,084"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754"
...,...,...,...,...,...
5938,Unknown,Red 11,"$7,000",$0,$0
5939,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5940,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5941,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


Things we want to look at.
1) Day of release affects profit
2) Genre of movie affects profit
3) Actors in movie affects profit
4) Season of release affects profit
5) international sales
6) average number of movies to make
7) number of reviews and gross #scatter plot




1) Genre of movie affects profit
    a) season for the genre to be released
    boxplot
    b) What genres succeed internationally
    boxplot x- genres, y profit
2) Number of reviews and profit
    a)scatter plot, x is number of votes,y is profit #see if number of views is what matters
        i) color code the review rating, 10-8 green, 7-5 yellow etc...
    b) scatter plot, x is number of votes, y is production cost #see if higher production costs get more views
3) How does the number of movies created by a studio affect the average profit
    a)do franchises do better

Starting to look at and scrub the data. The first step is going to be converting the datatypes into a usable form. 

In [3]:
df = pd.read_csv('zippedData/movieMoney.csv', index_col=0)

In [109]:
df.dtypes

Release_Date         datetime64[ns]
Movie_Name                   string
Production_Budget             int32
Gross_Domestic                int32
Gross_Worldwide               int64
dtype: object

In [3]:
df['Movie_Name'] = df['Movie_Name'].astype('string')

df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors='coerce')

df['Production_Budget'] = df['Production_Budget'].str.replace('$', '')
df['Production_Budget'] = df['Production_Budget'].str.replace(',', '')
df['Production_Budget'] = df['Production_Budget'].astype(int)

df['Gross_Domestic'] = df['Gross_Domestic'].str.replace('$', '')
df['Gross_Domestic'] = df['Gross_Domestic'].str.replace(',', '')
df['Gross_Domestic'] = df['Gross_Domestic'].astype(int)

df['Gross_Worldwide'] = df['Gross_Worldwide'].str.replace('$', '')
df['Gross_Worldwide'] = df['Gross_Worldwide'].str.replace(',', '')
df['Gross_Worldwide'] = df['Gross_Worldwide'].astype('int64')

df.sort_values('Release_Date', inplace=True, ascending=False) #sort by release date, most recent on bottom
df.head()

,Release_Date,Movie_Name,Production_Budget,Gross_Domestic,Gross_Worldwide
1502,2022-12-16,Heaven and Hell,40000000,0,0
602,2021-01-15,355,75000000,0,0
108,2021-01-01,Moonfall,150000000,0,0
133,2020-12-23,Top Gun: Maverick,140000000,0,0
20,2020-11-12,No Time to Die,250000000,0,0


The second step is to remove the films from the dataframe that haven't happened yet. The next step is to count the number of films that have no budget or not profit to see if they can be removed from the dataset without any problems.

The next step is to go through the IMDB dataset to remove data that we don't have profit information for.

Here we are loading in the dataset that we will be comparing the dataframe we have money information from. We're going to go ahead and convert the datatypes to their usable forms while we're doing this. 

In [9]:
title_basics_df = pd.read_csv('zippedData/title.basics.tsv', sep='\t')
title_basics_list = list(title_basics_df.columns)
unwanted_col = ['isAdult', 'startYear', 'endYear', 'runtimeMinutes']
title_basics_list = [ele for ele in title_basics_list if ele not in unwanted_col] 
for col in title_basics_list:
    title_basics_df[col] = title_basics_df[col].astype('string')
title_basics_df.drop(title_basics_df[title_basics_df['titleType'] != 'movie'].index, inplace=True) #drop any rows that aren't movies
title_basics_df['startYear'] = pd.to_datetime(title_basics_df['startYear'], errors='coerce')
title_basics_df['endYear'] = pd.to_datetime(title_basics_df['endYear'], errors='coerce')
title_basics_df['runtimeMinutes'].replace(to_replace='\\N', value='0', inplace=True)
title_basics_df['runtimeMinutes'] = title_basics_df['runtimeMinutes'].astype('int', errors='ignore')

In [10]:
movie_list = list(df['Movie_Name'].unique()) #create a list of the movie titles we have money information for
title_basics_df.drop(title_basics_df[title_basics_df['titleType'] != 'movie'].index, inplace=True) #drop any rows that are not movies
title_basics_df = title_basics_df[title_basics_df['primaryTitle'].isin(movie_list)] #only keep movies we have money information for
movie_list = list(title_basics_df['tconst'].unique()) #making the new list of movie names with IMDB uid
title_basics_df.to_csv('zippedData/title_basics_pared.csv')
title_basics_df.head() #preview what we got
title_basics_df = None #clear the variable for memory, was having memory issues and I have a lot of DF to go through

We now have a dataframe that only contains movies we have profit information for. We can now use the unique identifier for movies titles in IMDB's database to continue to pare down the remaining dataframes that we have. 

In [8]:
title_crew_df = pd.read_csv('zippedData/title.crew.tsv', sep='\t')
title_crew_df = title_crew_df[title_crew_df['tconst'].isin(movie_list)] #only keep movies we have money information for
title_crew_df.to_csv('zippedData/title_crew_pared.csv')
title_crew_df.head() #preview what we got
title_crew_df = None

In [7]:
title_akas_df = pd.read_csv('zippedData/title.akas.tsv', sep='\t')
title_akas_df = title_akas_df[title_akas_df['titleId'].isin(movie_list)] #only keep movies we have money information for
title_akas_df.to_csv('zippedData/title_akas_pared.csv')
title_akas_df.head() #preview what we got
title_akas_df = None

D:\Python\anaconda37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#title_principals_df = pd.read_csv('zippedData/title.principals.tsv', sep='\t')
iter_csv = pd.read_csv('zippedData/title.principals.tsv', sep='\t', iterator=True, chunksize=10000)
title_principals_df = pd.concat([chunk[chunk['tconst'].isin(movie_list)] for chunk in iter_csv])
title_principals_df.to_csv('zippedData/title_principals_pared.csv')
title_principals_df.head() #preview what we got
title_principals_df = None

In [6]:
title_ratings_df = pd.read_csv('zippedData/title.ratings.tsv', sep='\t')
title_ratings_df = title_ratings_df[title_ratings_df['tconst'].isin(movie_list)] #only keep movies we have money information for
title_ratings_df.to_csv('zippedData/title_ratings_pared.csv')
title_ratings_df.head() #preview what we got
title_ratings_df = None

In [12]:
name_basics_df = pd.read_csv('zippedData/name.basics.tsv', sep='\t')
name_basics_df['nconst'] = name_basics_df['nconst'].astype('string')
name_basics_df['primaryName'] = name_basics_df['primaryName'].astype('string')
name_basics_df['primaryProfession'] = name_basics_df['primaryProfession'].astype('string')
name_basics_df['knownForTitles'] = name_basics_df['knownForTitles'].astype('string')
name_basics_df['birthYear'] = pd.to_datetime(name_basics_df['birthYear'], errors='coerce')
name_basics_df['deathYear'] = pd.to_datetime(name_basics_df['deathYear'], errors='coerce')
name_basics_df.dtypes

nconst                       string
primaryName                  string
birthYear            datetime64[ns]
deathYear            datetime64[ns]
primaryProfession            string
knownForTitles               string
dtype: object

In [17]:
title_akas_df.drop(title_akas_df[title_akas_df['region'] != 'US'].index, inplace=True) #holding onto this code incase I want it

In [17]:
title_akas_df = pd.read_csv('zippedData/title_akas_pared.csv', index_col=0)
title_akas_df['isOriginalTitle'].replace(to_replace='\\N', value='1', inplace=True)
title_akas_df['titleId'] = title_akas_df['titleId'].astype('string')
title_akas_df['title'] = title_akas_df['title'].astype('string')
title_akas_df['region'] = title_akas_df['region'].astype('string')
title_akas_df['language'] = title_akas_df['language'].astype('string')
title_akas_df['types'] = title_akas_df['types'].astype('string')
title_akas_df['attributes'] = title_akas_df['attributes'].astype('string')
title_akas_df['isOriginalTitle'] = title_akas_df['isOriginalTitle'].astype('int')
title_akas_df['titleId'] = title_akas_df['titleId'].astype('string')
title_akas_df.dtypes

titleId            string
ordering            int64
title              string
region             string
language           string
types              string
attributes         string
isOriginalTitle     int32
dtype: object

In [21]:
title_crew_df = pd.read_csv('zippedData/title_crew_pared.csv', index_col=0)
title_crew_list = list(title_crew_df.columns)
for col in title_crew_list:
    title_crew_df[col] = title_crew_df[col].astype('string')
title_crew_df.dtypes

In [29]:
title_principals_df = pd.read_csv('zippedData/title_principals_pared.csv', index_col=0)
title_principals_list = list(title_principals_df.columns)
title_principals_list.remove('ordering')
for col in title_principals_list:
    title_principals_df[col] = title_principals_df[col].astype('string')
title_principals_df.dtypes

In [34]:
title_ratings_df = pd.read_csv('zippedData/title_ratings_pared.csv', index_col=0)
title_ratings_df['tconst'] = title_ratings_df['tconst'].astype('string')
title_ratings_df.dtypes

I've created all the dataframes that I need to use for the project. I now need to look through them and combine them in a meaningful way. A SQL database could work here, or a pivot table. 

In [ ]:
name_basics_df = pd.read_csv('zippedData/name.basics.tsv', sep='\t')
title_akas_df = pd.read_csv('zippedData/title.akas.tsv', sep='\t')
title_basics_df = pd.read_csv('zippedData/title.basics.tsv', sep='\t')
title_crew_df = pd.read_csv('zippedData/title.crew.tsv', sep='\t')
title_principals_df = pd.read_csv('zippedData/title.principals.tsv', sep='\t')
title_ratings_df = pd.read_csv('zippedData/title.ratings.tsv', sep='\t')

1) Genre of movie affects profit a) season for the genre to be released boxplot b) What genres succeed internationally boxplot x- genres, y profit

In [ ]:
sns.boxplot(x='genre', y='profit', hue='international?', data=df)

In [ ]:
sns.boxplot(x='season', y='profit', hue='international?',
            col='genre', col_wrap=6, data=df)
#idk depends on the number of genres really

Plot to answer question 2?
Seems too easy.
2) Number of reviews and profit a)scatter plot, x is number of votes,y is profit #see if number of views is what matters i) color code the review rating, 10-8 green, 7-5 yellow etc... b) scatter plot, x is number of votes, y is production cost #see if higher production costs get more views.

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.lmplot(x='num_votes', y='profit', hue='rating', data=df)

In [ ]:
sns.lmplot(x='num_votes', y='prod_cost', hue='rating', data=df)
#wait this should be reversed. Does higher production cost lead to higher  votes
#not do movies with more votes have higher votes

#instead of using hue as rating, could make x amount different plots
#separating the ratings into the same number of groups
#sns.lmplot(x='num_vote', y='prod_cost', col='rating', data=df)

3) How does the number of movies created by a studio affect the average profit a)do franchises do better

I need the dataset to actually do this...
idk this seems like too few plots, or not ambitious enough. We're only going to have like 15 plots total at this rate. 

In [ ]:
sns.boxplot(x='num_movies_inyear', y='avg_profit', hue='franchise?', data=df)
#this question is going to be closer to
#for each studio, group them by the number of films that they put out in a year
#and then determine what the average profit is for each group
#depending on the number of groups, may need to bin them.
#then, subdivide those groups into films that are part of franchises
#and those that are not